# Car racing game agent evolving through iterations of NEAT algorithm


**Import of the different functions defined to build a track and move the car around it**

In [14]:
using Plots
using Random

We use a mutable structure ${track}$ which is an array of Int64. It is initialized with only zeros representing blank parts.

In [15]:
mutable struct track
    road::Array{Int64}
end

function track(n::Int64)
    track(zeros(n,n))
end

track

We then define several functions dictating the movement of the car around the track.

In [16]:
function right!(tr::track,pos::Int64,dir::Int64,larg::Int64,h::Int64)
    r = tr.road
    dir2 = dir%4
    if dir2 == 0
        for i in 0:larg-1
            r[pos + i*n : pos + (larg-1) + i*n] = [h+larg - j for j in 1:larg] #ones(larg)*(h+larg-i)
        end
        pos = pos-1
    end
    if dir2 == 1
        for i in 0:larg-1
            r[pos - i*n : pos + (larg-1) - i*n] = ones(larg)*(h+larg-i)
        end
        pos = pos+n
    end
    
    if dir2 == 2
        for i in 0:larg-1
            r[pos - i*n - (larg-1): pos - i*n] = [j for j in h:h+larg-1] #ones(larg)*(h+larg-i)
        end
        pos = pos+1
    end
    
    if dir2 == 3
        for i in 0:larg-1
            r[pos + i*n - (larg-1): pos + i*n] = ones(larg)*(h+larg-i)
        end
        pos = pos-n
    end
    
    r[pos]=h
    dir = dir-1
    
    (pos,dir)
end


function left!(tr::track,pos::Int64,dir::Int64,larg::Int64,h::Int64)
    r = tr.road
    dir2 = dir%4
    if dir2 == 0
        for i in 0:larg-1
            r[pos + i*n : pos + (larg-1) + i*n] = [j for j in h:h+larg-1] #ones(larg)*(h+i)
        end
        pos = pos+(larg)*(n+1)-n
    end
    
    if dir2 == 1
        for i in 0:larg-1
            r[pos - i*n : pos + (larg-1) - i*n] = ones(larg)*(h+i)
        end
        pos = pos-(larg)*(n-1)-1
    end
    
    if dir2 == 2
        for i in 0:larg-1
            r[pos - i*n - (larg-1): pos - i*n] = [h+larg - j for j in 1:larg] # ones(larg)*(h+i)
        end
        pos = pos-(larg)*(n+1)+n
    end
    
    if dir2 == 3
        for i in 0:larg-1
            r[pos + i*n - (larg-1): pos + i*n] = ones(larg)*(h+i)
        end
        pos = pos+(larg)*(n-1)+1
    end
    
    
    r[pos]=h
    dir = dir+1
    
    (pos,dir)
end

function droit!(tr::track,pos::Int64,dir::Int64,larg::Int64,long::Int64,h::Int64)
    r = tr.road
    dir2 = dir%4

    if dir2 == 0
        for i in 0:long-1
            r[pos + i*n : pos + (larg-1) + i*n] = ones(larg)*h
        end
        pos = pos+(long)*n
    end
    
    if dir2 == 1
        for i in 0:larg-1
            r[pos - i*n : pos - i*n + (long-1)] = ones(long)*h
        end
        pos = pos+long
    end
    
    if dir2 == 2
        for i in 0:long-1
            r[pos - i*n - (larg-1) : pos - i*n] = ones(larg)*h
        end
        pos = pos-(long)*n
    end
    
    if dir2 == 3
        for i in 0:larg-1
            r[pos + i*n - (long-1) : pos + i*n] = ones(long)*h
        end
        pos = pos-long
    end
    
    r[pos]=h
    (pos,dir)
end

function long!(tr::track,pos::Int64,dir::Int64,larg::Int64,h::Int64)
    (pos,dir) = short!(tr,pos,dir,larg,h)
    (pos,dir) = short!(tr,pos,dir,larg,h+3)
    (pos,dir) = short!(tr,pos,dir,larg,h+6)
    (pos,dir) = short!(tr,pos,dir,larg,h+9)
end

function short!(tr::track,pos::Int64,dir::Int64,larg::Int64,h::Int64)
    (pos,dir) = droit!(tr,pos,dir,larg,1,h)
    (pos,dir) = droit!(tr,pos,dir,larg,1,h+1)
    (pos,dir) = droit!(tr,pos,dir,larg,1,h+2)
end

short! (generic function with 1 method)

A function ${build}$ which builds the entire track.

In [17]:
function build!(tr::track,L::Array{Int64})
    pos = 5010
    dir = 100
    tr.road[5001-n] = 1
    
    h = 100
    for i in L
        if i == 1
            (pos,dir) = short!(tr,pos,dir,larg,h)
        end
        
        if i == 2
            (pos,dir) = long!(tr,pos,dir,larg,h)
            h = h+3
        end
        
        if i == 3
            (pos,dir) = right!(tr,pos,dir,larg,h)
            h = h+larg
        end
        
        if i == 4
            (pos,dir) = left!(tr,pos,dir,larg,h)
            h = h+larg
        end
        h = h + 1
    end
end

R1 = track(n)
L = [1,2,4,2,3,4,2,2,2,4,2,2,2,2,4,3,4,1,4,2,2,3,2,2,1,2,4]


build!(R1,L)
plot(heatmap(R1.road),size=(700,600))

UndefVarError: UndefVarError: larg not defined

We then define our track used for comparison and as an example.

In [18]:
n = 100

track1 = track(n)
L = [1,2,4,2,3,4,2,2,2,4,2,2,2,2,4,3,4,1,4,2,2,3,2,2,1,2,4]


build!(track1,L)
plot(heatmap(track1.road),size=(700,600))

UndefVarError: UndefVarError: larg not defined

Then our agent ${car}$ with the following attributes:

In [19]:
mutable struct car
    x::Float64
    y::Float64
    v::Float64
    angle::Float64
    a::Float64
    braq::Float64
    vmax::Float64
end

function car(x::Float64,y::Float64,angle::Float64)
    car(x,y,0,angle,5,3,20) #angle CI
end

car

In [20]:
function move!(Car::car,input::Int64,dt::Float64)
    delta = Car.v*dt
    
    if input == 1 || input == 2 || input == 8
        if Car.v<Car.vmax
            delta = delta + 0.5*Car.a*(dt^2)
            Car.v = Car.v + Car.a*dt
        end
    end
    
    if input == 4 || input == 5 || input == 6
        if Car.v>0
            delta = delta - 0.5*Car.a*(dt^2)
            Car.v = Car.v - Car.a*dt
        end
    end
    
    if input == 2 || input == 3 || input == 4
        braqv = Car.braq*(1 + 3*Car.v/Car.vmax)
        Car.angle = Car.angle - delta/braqv
    end
    
    if input == 6 || input == 7 || input == 8
        braqv = Car.braq*(1 + 3*Car.v/Car.vmax)
        Car.angle = Car.angle + delta/braqv
    end
    
    Car.x = Car.x + cos(Car.angle)*delta
    Car.y = Car.y + sin(Car.angle)*delta
end

move! (generic function with 1 method)

## NEAT algorithm

We need first to define the different parameters we will be using when defining our different functions.

Thus, the following parameters and values are used as a reference:

In [21]:
mutable struct Node
    
    node_id::Int64
    node_type::String
    #"input", "output" or "hidden"
end

In [31]:
function Node(node_id::Int64, node_type::String)
    
    Node(node_id, node_type)
end

Node

In [30]:
mutable struct Connection
    
    input_node::Node
    output_node::Node
    weight::Float64
    status::Bool
    #if status is false, then the connection is disabled
    innovation_number::Int64
    #this last parameter will allow historical marking
end

In [28]:
function Connection(input_node::Node, output_node::Node, weight::Float64, innovation_number::Float64)
    
    Connection(input_node, output_node, weight, status=true, innovation_number)
end

Connection

In [25]:
mutable struct Network
    
    node_genes::Array{Node}
    node_connections::Array{Connection}
end

In [26]:
mutable struct NEAT
    #mr stands for mutation rate
    mr_change_weight::Float64
    
    #if a change of weight occurs, the following rates are applied
    mr_change_weight_uniform::Float64
    mr_change_weight_random::Float64
    #the previous mr is equal to 1-mr_change_weight_uniform

    mr_add_node::Float64
    mr_add_connection::Float64
    
    percentage_no_co::Float64
    #percentage of the population on which mutations are directly applied without the crossover step
    
    #parameters measuring the distance between two topologies
    c1::Float64
    c2::Float64
    c3::Float64
    threshold::Float64
    
    population::Array{Array{Network}}
    offspring::Array{Array{Network}}
end

In [35]:
function NEAT(pop_size=150::Int64, mr_change_weight=0.8::Float64, mr_change_weight_uniform=0.9::Float64, mr_add_node=0.03::Float64, 
        mr_add_connection=0.05::Float64, percentage_no_co=0.25::Float64, c1=1.0::Float64, c2=1.0::Float64, c3=0.4::Float64, threshold=3.0::Float64)
    
    population = Array{Array{Network}}[pop_size]
    offspring = Array{Array{Network}}[pop_size]
    
    NEAT(mr_change_weight, mr_change_weight_uniform, 1.0-mr_change_weight_uniform, mr_add_node, mr_add_connection, percentage_no_co, 
        c1, c2, c3, threshold, population, offspring)
end

NEAT